# Ejemplo: Comparando modelos utilizando 5x2 cross-validation

## Introducción

Instalamos la librerias necesarias

In [ ]:
!wget https://raw.githubusercontent.com/santiagxf/E72102/master/docs/develop/modeling/selection/code/5x2.txt \
    --quiet --no-clobber
!pip install -r 5x2.txt --quiet

### Sobre el conjunto de datos del censo UCI

El conjunto de datos del censo de la UCI es un conjunto de datos en el que cada registro representa a una persona. Cada registro contiene 14 columnas que describen a una una sola persona, de la base de datos del censo de Estados Unidos de 1994. Esto incluye información como la edad, el estado civil y el nivel educativo. La tarea es determinar si una persona tiene un ingreso alto (definido como ganar más de $50 mil al año). Esta tarea, dado el tipo de datos que utiliza, se usa a menudo en el estudio de equidad, en parte debido a los atributos comprensibles del conjunto de datos, incluidos algunos que contienen tipos sensibles como la edad y el género, y en parte también porque comprende una tarea claramente del mundo real.

Preparando nuestros conjuntos de datos

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('datasets/uci_census/data/adult-train.csv')

train = df.drop(['income'], axis=1)
target = df['income'].to_numpy()

## Preparación de los datos para el ejemplo

Realizaremos un pequeño preprocesamiento antes de entrenar el modelo:

- Imputaremos los valores faltantes de las caracteristicas numéricas con la media
- Imputaremos los valores faltantes de las caracteristicas categóricas con el valor `?`
- Escalaremos los valores numericos utilizando un `StandardScaler`
- Codificaremos las variables categóricas utilizando `OneHotEncoder`

In [2]:
from typing import Tuple, List

import sklearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer


def prepare(X: pd.DataFrame) -> Tuple[np.ndarray, sklearn.compose.ColumnTransformer]:
    pipe_cfg = {
        'num_cols': X.dtypes[X.dtypes == 'int64'].index.values.tolist(),
        'cat_cols': X.dtypes[X.dtypes == 'object'].index.values.tolist(),
    }
    
    num_pipe = Pipeline([
        ('num_imputer', SimpleImputer(strategy='median')),
        ('num_scaler', StandardScaler())
    ])
    
    cat_pipe = Pipeline([
        ('cat_imputer', SimpleImputer(strategy='constant', fill_value='?')),
        ('cat_encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))
    ])
    
    transformations = ColumnTransformer([
        ('num_pipe', num_pipe, pipe_cfg['num_cols']),
        ('cat_pipe', cat_pipe, pipe_cfg['cat_cols'])
    ])
    X = transformations.fit_transform(X)
    
    return X, transformations


train, transformations = prepare(train)

## Definiendo nuestros modelos a comparar

Para demostrar la técnica, utilizaremos dos clasificadores basados en `LightGBM`

In [3]:
from lightgbm import LGBMClassifier

clf1 = LGBMClassifier(n_estimators=100, n_jobs=2)
clf2 = LGBMClassifier(n_estimators=100, reg_alpha=1, reg_lambda=1, min_split_gain=2, n_jobs=2)

## Procedimiento de 5x2

Generamos las semillas con las cuales generar los conjuntos de datos, los cuales son 5:

In [4]:
import random

seeds = [ random.randint(1,10000) for time in range(0,5)]

Inicializamos arreglos para guardar los valores necesarios:

In [5]:
p = np.zeros((5,2)) # (numero de folds, numero de iteraciones)
scores = np.zeros((2,5,2)) # (numero de modelos, numero de folds, numero de iteraciones)
diff_scores = np.zeros((5,2)) # (numero de folds, numero de iteraciones)
s_sqr = 0

El siguiente procedimiento calcula el valor estadístico que estamos necesitando:

In [6]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score

for i, seed in enumerate(seeds):
    # Split the dataset in 2 parts with the current seed
    folds = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
    
    # Initialize score differences
    for k, (trn_idx, val_idx) in enumerate(folds.split(target, target)):
        # Split the data
        trn_x, trn_y = train[trn_idx], target[trn_idx]
        val_x, val_y = train[val_idx], target[val_idx]
        
        # Train classifiers
        clf1.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], early_stopping_rounds=20, verbose=0)
        clf2.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], early_stopping_rounds=20, verbose=0)
        
        # Compute scores
        preds_1 = clf1.predict_proba(val_x, num_iteration=clf1.best_iteration_)[:, 1]
        scores[0][i][k] = roc_auc_score(val_y, preds_1)
        
        preds_2 = clf2.predict_proba(val_x, num_iteration=clf2.best_iteration_)[:, 1]
        scores[1][i][k] = roc_auc_score(val_y, preds_2)
        
        diff_scores[i][k] = scores[0][i][k] - scores[1][i][k]
        print("Fold %2d score difference = %.6f" % (k + 1, diff_scores[i][k]))
        
        # Compute score difference for current fold
        p[i][k] = diff_scores[i][k]
        

    # Compute mean of scores difference for the current 2-fold CV
    p_i_bar = (p[i][0] + p[i][1]) / 2
    # Compute the variance estimate for the current 2-fold CV
    s_sqr_i = (p[i][0] - p_i_bar) ** 2 + (p[i][1] - p_i_bar) ** 2 
    # Add up to the overall variance
    s_sqr += s_sqr_i
    
# Compute t value as the first difference divided by the square root of variance estimate
t_bar = p[0][0] / ((s_sqr / 5) ** .5) 

print("Performance del clasificador 1: %.6f +/- %.6f" % (np.mean(scores[0]), np.std(scores[0])))
print("Performance del clasificador 2 : %.6f +/- %.6f" % (np.mean(scores[1]), np.std(scores[1])))
print("Diferencia entre las performance: %.6f +/- %.6f" % (np.mean(diff_scores), np.std(diff_scores)))
print("Valor estadístico de t: %.6f" % t_bar )

Fold  1 score difference = 0.002469
Fold  2 score difference = 0.002163
Fold  1 score difference = 0.001980
Fold  2 score difference = 0.001963
Fold  1 score difference = 0.001847
Fold  2 score difference = 0.001401
Fold  1 score difference = 0.001902
Fold  2 score difference = 0.001199
Fold  1 score difference = 0.002051
Fold  2 score difference = 0.001228
Performance del clasificador 1: 0.925682 +/- 0.001267
Performance del clasificador 2 : 0.923862 +/- 0.001479
Diferencia entre las performance: 0.001820 +/- 0.000395
Valor estadístico de t: 6.450622


Computamos los valores críticos del intervalo de confianza para la distribución, significancia y grados de libertad:

In [7]:
from scipy.stats import t

interval = t.interval(0.95, 5)

Tomamos una decisión

In [8]:
if t_bar > interval[0] and t_bar < interval[1]:
    print("No podemos tomar ninguna conclusión. No se puede rechazar la idea de que ambos modelos son equivalente")
else:
    print("Existe suficiente evidencia para rechazar la idea de que los modelos son equivalentes en favor de \
          una alternativa de que los modelos son distintos.")

Existe suficiente evidencia para rechazar la idea de que los modelos son equivalentes en favor de           una alternativa de que los modelos son distintos.
